In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# # for gmail
%cd /content/drive/MyDrive/ODU/Research_ODU/R001_Fall_2023_AI_Reproducibility/model_training
!ls

# # for odu
# %cd /content/drive/MyDrive/ODU/Research_Work/R001_Fall_2023_AI_Reproducibility/model_training
# !ls


/content/drive/MyDrive/ODU/Research_ODU/R001_Fall_2023_AI_Reproducibility/model_training
automated_labelling
dataset
labelled_citing_papers
lemos_R_001_M6_sentiment_predictor_001_12ep_bert.keras
lemos_R_001_sentiment_binary_002_15ep_bert.keras
model_0_predictions
model_0_predictions_for_all_42000
model_6_predictions_related_only
plots
R_001_2_label_binary_sentiment_classification_model_training.ipynb
R_001_3_label_multiclass_sentiment_classification_model_training.ipynb
R_001_binary_text_classification_related_not_related.ipynb
R_001_M6_3_class_sentiment_classification.ipynb
R_001_M7_1_binary_classification_related_not_related.ipynb
R_001_M7_2_binary_sentiment_classification.ipynb
R_001_multiclass_text_classification.ipynb
R_001_Sentiment_Analysis.ipynb
R_001_wahab.rdp


In [6]:
!pip install keras-core --upgrade
!pip install -q keras-nlp --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 15.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatible

In [2]:
# -*- coding: utf-8 -*-
"""
R_001_Research_Project__
|
R_001_M7_2_binary_sentiment_classification.ipynb
Created on Thu Jan 18 16:30:03 2024
@author: Rochana Obadage
"""

import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_core as keras
import keras_nlp

import pytz
from datetime import datetime
import re
import argparse
import sys


Using TensorFlow backend


In [ ]:
!python3 -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"

2024-02-07 06:43:29.085414: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-07 06:43:29.085453: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-07 06:43:29.086562: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-07 06:43:30.220583: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[]


# Load the datasets

In [8]:
mapping = {'negative':0,'positive':1}

In [12]:
df_dataset = pd.read_csv(r"../data/1937_dataset_for_3_label_sentiment.csv")
df_dataset = df_dataset[df_dataset['3_label']!='neutral']
df_dataset


df_dataset['M72_target'] = df_dataset['3_label'].map(mapping)
df_dataset = df_dataset.reset_index(drop=True)
df_dataset

,context,label_score,label,3_label,target,M72_target
0,The proposed method can be seamlessly implante...,0.5,Weak,positive,2,1
1,To further demonstrate the effectiveness of Bi...,0.5,Weak,positive,2,1
2,"focusing on the feature fusion [39], [49] with...",0.5,Weak,positive,2,1
3,"To improve embedding quality, detaching class-...",0.5,Weak,positive,2,1
4,We compared the performance of our likelihood ...,0.5,Weak,positive,2,1
...,...,...,...,...,...,...
176,"To begin, we apply our table extraction module...",0.5,Weak,positive,2,1
177,Paper metadata from these sources are harmoniz...,0.5,Weak,positive,2,1
178,This tool is available online.12 For table ext...,0.5,Weak,positive,2,1
179,The source codes of 6 papers were not executab...,-2.0,O-NR,negative,0,0


In [13]:
mapping_reverse = dict((value,key) for key,value in mapping.items())
mapping_reverse

{0: 'negative', 1: 'positive'}

In [14]:
# label counts per each category
df_ = df_dataset.groupby('3_label').count()
df_.reset_index()

,3_label,context,label_score,label,target,M72_target
0,negative,23,23,23,23,23
1,positive,158,158,158,158,158


In [15]:
df_dataset_positive = df_dataset[df_dataset['3_label'] =='positive'].sample(n=23, replace=False)
df_dataset_negative = df_dataset[df_dataset['3_label'] =='negative']

df_balanced = pd.concat([df_dataset_positive,df_dataset_negative], ignore_index = True)
df_balanced = df_balanced.sample(frac = 1).reset_index(drop=True)
df_balanced
# df_balanced.to_csv(r'dataset/1937_dataset_for_3_label_sentiment_test_balanced_93.csv',index=False)

,context,label_score,label,3_label,target,M72_target
0,"For an input x, the augmented views a1,a2 ∈ Rp...",0.5,Weak,positive,2,1
1,This method outperformed DeepDeSRT and TableNe...,0.5,Weak,positive,2,1
2,"However, we were unable to obtain reasonable p...",-2.0,O-NR,negative,0,0
3,The source codes of 6 papers were not executab...,-2.0,O-NR,negative,0,0
4,"Also, [23, 26] used different train/test split...",-1.0,P-NR,negative,0,0
5,The source codes of 6 papers were not executab...,-2.0,O-NR,negative,0,0
6,"Furthermore, 98 looking at the official datase...",1.0,Strong,positive,2,1
7,"Under the new evaluation setting, the result o...",-2.0,O-NR,negative,0,0
8,"Interestingly, Variation 15, with a shear hori...",0.5,Weak,positive,2,1
9,We unfortunately could not directly evaluate t...,-1.0,P-NR,negative,0,0


In [16]:
df_b = df_balanced.groupby('3_label').count()
df_b.reset_index()

,3_label,context,label_score,label,target,M72_target
0,negative,23,23,23,23,23
1,positive,23,23,23,23,23


In [17]:
# lable names
print(set(df_dataset['3_label']))
print(set(df_dataset['M72_target']))

{'negative', 'positive'}
{0, 1}


# Load a DistilBERT model from Keras NLP


In [19]:
BATCH_SIZE = 16
NUM_TRAINING_EXAMPLES = df_balanced.shape[0]
TRAIN_SPLIT = 1
VAL_SPLIT = 0
STEPS_PER_EPOCH = int(NUM_TRAINING_EXAMPLES)*TRAIN_SPLIT // BATCH_SIZE

EPOCHS = 15 #12
AUTO = tf.data.experimental.AUTOTUNE
NUM_TRAINING_EXAMPLES

46

In [20]:
# Load a DistilBERT model.
preset= "distil_bert_base_en_uncased"

# Use a shorter sequence length.
preprocessor = keras_nlp.models.DistilBertPreprocessor.from_preset(preset,
                                                                   sequence_length=512,
                                                                   )
# preprocessor_4_tweets
# Pretrained classifier.
classifier = keras_nlp.models.DistilBertClassifier.from_preset(preset,
                                                               preprocessor = preprocessor,
                                                               num_classes=2)

# vgg_model.add(tf.keras.layers.Dense(10, activation = 'softmax'))
classifier.summary()




100%|██████████| 580/580 [00:00<00:00, 106kB/s]
100%|██████████| 226k/226k [00:00<00:00, 993kB/s] 
100%|██████████| 515/515 [00:00<00:00, 289kB/s]
100%|██████████| 253M/253M [00:06<00:00, 44.2MB/s]


Preprocessor: "distil_bert_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ distil_bert_tokenizer (DistilBertTokenizer)        │                                              30,522 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "distil_bert_classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ distil_bert_backbone          │ (None, None, 768)         │      66,362,880 │ padding_mask[0][0],        │
│ (DistilBertBackbone)          │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ get_item (GetItem)            │ (None, 768)               │               0 │ distil_bert_backbone[0][0] │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pooled_dense (Dense)          │ (None, 768)               │         590,592 │ get_item[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ output_dropout (Dropout)      │ (None, 768)               │               0 │ pooled_dense[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ logits (Dense)                │ (None, 2)                 │           1,538 │ output_dropout[0][0]       │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 66,955,010 (255.41 MB)

 Trainable params: 66,955,010 (255.41 MB)

 Non-trainable params: 0 (0.00 B)

# Train your own model, fine-tuning BERT

In [ ]:
from sklearn.model_selection import KFold
from keras_nlp.models import DistilBertClassifier
from sklearn.metrics import classification_report
import numpy as np


fold_accuracies = []
fold_reports = []
fold_matrices = []

df_train_for_5_fold = df_balanced.copy()

kfold = KFold(n_splits=5, shuffle=True)

for train_index, test_index in kfold.split(df_train_for_5_fold):

    # Get the training and test data for the current fold
    X_train, X_test = df_train_for_5_fold.iloc[train_index], df_train_for_5_fold.iloc[test_index]
    y_train, y_test = X_train['M72_target'], X_test['M72_target']


    classifier = keras_nlp.models.DistilBertClassifier.from_preset(preset,
                                                               preprocessor = preprocessor,
                                                               num_classes=2)

    # uncomment below if error
    # classifier.fit(x=X_train, y=y_train, batch_size=2)

    # Re-compile (e.g., with a new learning rate)
    classifier.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(1e-5),
        metrics= ["accuracy"],
    )

    history = classifier.fit(x=X_train['context'],
                            y=y_train,
                            batch_size=BATCH_SIZE,
                            epochs=EPOCHS,
                            )



    # Evaluate the model on the test data
    y_pred = classifier.predict(X_test['context'])


    # Calculate the accuracy
    accuracy = np.mean(np.argmax(y_pred, axis=1) == y_test)
    print(f"Accuracy: {accuracy}")

    print('\n\n')
    # label_to_score = {'related':1,"not_related":0}
    print('classifiation report : ')
    print(classification_report(y_test, np.argmax(y_pred, axis=1),target_names= mapping.keys()))


    # Calculate the accuracy
    accuracy = np.mean(np.argmax(y_pred, axis=1) == y_test)
    fold_accuracies.append(accuracy)

    # Generate classification report
    report = classification_report(y_test, np.argmax(y_pred, axis=1),target_names= mapping.keys())
    # report = classification_report(y_test, y_pred, output_dict=True)
    fold_reports.append(report)

    # # Generate confusion matrix
    matrix = confusion_matrix(y_test, np.argmax(y_pred, axis=1))
    fold_matrices.append(matrix)

# Print the average accuracy across all folds
print("Average Accuracy:", np.mean(fold_accuracies))

# Print the classification reports for each fold
for i, report in enumerate(fold_reports):
    print(f"Fold {i+1} Classification Report:")
    print(report)

# Print the confusion matrices for each fold
for i, matrix in enumerate(fold_matrices):
    print(f"Fold {i+1} Confusion Matrix:")
    print(matrix)


/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids


Epoch 1/15
3/3 [==============================] - 21s 675ms/step - loss: 0.6869 - accuracy: 0.5556
Epoch 2/15
3/3 [==============================] - 2s 593ms/step - loss: 0.6730 - accuracy: 0.5556
Epoch 3/15
3/3 [==============================] - 2s 595ms/step - loss: 0.6589 - accuracy: 0.5556
Epoch 4/15
3/3 [==============================] - 2s 599ms/step - loss: 0.6459 - accuracy: 0.5833
Epoch 5/15
3/3 [==============================] - 2s 603ms/step - loss: 0.6245 - accuracy: 0.6667
Epoch 6/15
3/3 [==============================] - 2s 616ms/step - loss: 0.6191 - accuracy: 0.7500
Epoch 7/15
3/3 [==============================] - 2s 605ms/step - loss: 0.5817 - accuracy: 0.8889
Epoch 8/15
3/3 [==============================] - 2s 605ms/step - loss: 0.5598 - accuracy: 0.8889
Epoch 9/15
3/3 [==============================] - 2s 600ms/step - loss: 0.5235 - accuracy: 0.9722
Epoch 10/15
3/3 [==============================] - 2s 599ms/step - loss: 0.4833 - accuracy: 1.0000
Epoch 11/15
3/3 [=

1/1 [==============================] - 2s 2s/step
Accuracy: 0.0



classifiation report : Training
              precision    recall  f1-score   support

    negative       1.00      0.67      0.80         3
    positive       0.88      1.00      0.93         7

    accuracy                           0.90        10
   macro avg       0.94      0.83      0.87        10
weighted avg       0.91      0.90      0.89        10



/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids


Epoch 1/15
3/3 [==============================] - 20s 680ms/step - loss: 0.6944 - accuracy: 0.4595
Epoch 2/15
3/3 [==============================] - 2s 609ms/step - loss: 0.6658 - accuracy: 0.7027
Epoch 3/15
3/3 [==============================] - 2s 609ms/step - loss: 0.6478 - accuracy: 0.7568
Epoch 4/15
3/3 [==============================] - 2s 608ms/step - loss: 0.6259 - accuracy: 0.7838
Epoch 5/15
3/3 [==============================] - 2s 611ms/step - loss: 0.5984 - accuracy: 0.8108
Epoch 6/15
3/3 [==============================] - 2s 627ms/step - loss: 0.5643 - accuracy: 0.7568
Epoch 7/15
3/3 [==============================] - 2s 615ms/step - loss: 0.5412 - accuracy: 0.7838
Epoch 8/15
3/3 [==============================] - 2s 620ms/step - loss: 0.5183 - accuracy: 0.7838
Epoch 9/15
3/3 [==============================] - 2s 616ms/step - loss: 0.4768 - accuracy: 0.8649
Epoch 10/15
3/3 [==============================] - 2s 630ms/step - loss: 0.4235 - accuracy: 0.9459
Epoch 11/15
3/3 [=

...

Epoch 1/15
3/3 [==============================] - 24s 655ms/step - loss: 0.6966 - accuracy: 0.5405
Epoch 2/15
3/3 [==============================] - 2s 609ms/step - loss: 0.6739 - accuracy: 0.7027
Epoch 3/15
3/3 [==============================] - 2s 608ms/step - loss: 0.6647 - accuracy: 0.7568
Epoch 4/15
3/3 [==============================] - 2s 623ms/step - loss: 0.6526 - accuracy: 0.8378
Epoch 5/15
3/3 [==============================] - 2s 614ms/step - loss: 0.6291 - accuracy: 0.9189
Epoch 6/15
3/3 [==============================] - 2s 619ms/step - loss: 0.6125 - accuracy: 0.8108
Epoch 7/15
3/3 [==============================] - 2s 623ms/step - loss: 0.5670 - accuracy: 0.9459
Epoch 8/15
3/3 [==============================] - 3s 747ms/step - loss: 0.5451 - accuracy: 0.9189
Epoch 9/15
3/3 [==============================] - 2s 624ms/step - loss: 0.5061 - accuracy: 0.9189
Epoch 10/15
3/3 [==============================] - 2s 627ms/step - loss: 0.4607 - accuracy: 0.9459
Epoch 11/15
3/3 [=

/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids


Epoch 1/15
3/3 [==============================] - 26s 654ms/step - loss: 0.6971 - accuracy: 0.4054
Epoch 2/15
3/3 [==============================] - 2s 607ms/step - loss: 0.6785 - accuracy: 0.7297
Epoch 3/15
3/3 [==============================] - 2s 607ms/step - loss: 0.6649 - accuracy: 0.7027
Epoch 4/15
3/3 [==============================] - 2s 609ms/step - loss: 0.6556 - accuracy: 0.7838
Epoch 5/15
3/3 [==============================] - 2s 628ms/step - loss: 0.6391 - accuracy: 0.8108
Epoch 6/15
3/3 [==============================] - 2s 614ms/step - loss: 0.6265 - accuracy: 0.8378
Epoch 7/15
3/3 [==============================] - 2s 620ms/step - loss: 0.5990 - accuracy: 0.9189
Epoch 8/15
3/3 [==============================] - 2s 621ms/step - loss: 0.5613 - accuracy: 0.9730
Epoch 9/15
3/3 [==============================] - 2s 621ms/step - loss: 0.5407 - accuracy: 0.9459
Epoch 10/15
3/3 [==============================] - 2s 637ms/step - loss: 0.5000 - accuracy: 0.9730
Epoch 11/15
3/3 [=

/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids


Epoch 1/15
3/3 [==============================] - 22s 638ms/step - loss: 0.6912 - accuracy: 0.5946
Epoch 2/15
3/3 [==============================] - 2s 606ms/step - loss: 0.6798 - accuracy: 0.5405
Epoch 3/15
3/3 [==============================] - 2s 605ms/step - loss: 0.6601 - accuracy: 0.6757
Epoch 4/15
3/3 [==============================] - 2s 609ms/step - loss: 0.6511 - accuracy: 0.7838
Epoch 5/15
3/3 [==============================] - 2s 626ms/step - loss: 0.6389 - accuracy: 0.8649
Epoch 6/15
3/3 [==============================] - 2s 614ms/step - loss: 0.6085 - accuracy: 0.9189
Epoch 7/15
3/3 [==============================] - 2s 619ms/step - loss: 0.5797 - accuracy: 0.9459
Epoch 8/15
3/3 [==============================] - 2s 620ms/step - loss: 0.5513 - accuracy: 0.8919
Epoch 9/15
3/3 [==============================] - 2s 622ms/step - loss: 0.5203 - accuracy: 0.9189
Epoch 10/15
3/3 [==============================] - 2s 636ms/step - loss: 0.4681 - accuracy: 0.9189
Epoch 11/15
3/3 [=

In [ ]:
saved_model_path = 'lemos_R_001_sentiment_binary_002_15ep_bert.keras'
classifier.save(saved_model_path)


/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/task.py:47: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/task.py:47: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: 

In [7]:
reloaded_model_22 = tf.keras.models.load_model('lemos_R_001_sentiment_binary_002_15ep_bert.keras')

/usr/local/lib/python3.10/dist-packages/keras/src/saving/serialization_lib.py:727: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 209 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [ ]:
import glob
import os


FOLDER_NAME = r'../data/M72_predictions_for_only_12442/'

if not os.path.exists(FOLDER_NAME):
    os.makedirs(FOLDER_NAME)

label_file_list = [i for i in glob.glob("../data/related_binary_labelled/*.csv")]
len(label_file_list)
count = 0
sum = 0


for file_ in label_file_list:
    df_ = pd.read_csv(file_)

    predictions_df = df_.copy()

    predictions_df = predictions_df[predictions_df['target_predict_label']=='related'][['text','target_predict','target_predict_label']].reset_index(drop=True)

    sum += len(predictions_df)
    if len(predictions_df) == 0:
        continue

    predictions_df['target_model_72_predict'] = 0
    x_test_series = predictions_df['text']


    predictions = reloaded_model_22.predict(x_test_series)
    predictions_df["target_model_72_predict"] = np.argmax(predictions, axis=1)


    mapping_reverse = {0: 'negative', 1:'positive'}
    predictions_df['target_predict_model_72_label'] = predictions_df["target_model_72_predict"].map(mapping_reverse)

    full_filename = os.path.join(FOLDER_NAME, os.path.basename(file_))
    print(full_filename)

    predictions_df.to_csv(full_filename,index=False)
    print(full_filename + ' saved')
    print(count)
    count+=1
    print("\n\n\n")

print(sum)


2/2 ━━━━━━━━━━━━━━━━━━━━ 63s 24s/step
../data/M72_predictions_for_only_12442/RS_006_MLRC_2022_06.csv
../data/M72_predictions_for_only_12442/RS_006_MLRC_2022_06.csv saved
0




1/1 ━━━━━━━━━━━━━━━━━━━━ 17s 17s/step
../data/M72_predictions_for_only_12442/RS_014_MLRC_2022_14.csv
../data/M72_predictions_for_only_12442/RS_014_MLRC_2022_14.csv saved
1




1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step
../data/M72_predictions_for_only_12442/RS_032_MLRC_2022_32.csv
../data/M72_predictions_for_only_12442/RS_032_MLRC_2022_32.csv saved
2




1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
../data/M72_predictions_for_only_12442/RS_023_MLRC_2022_23.csv
../data/M72_predictions_for_only_12442/RS_023_MLRC_2022_23.csv saved
3




1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
../data/M72_predictions_for_only_12442/RS_036_MLRC_2022_36.csv
../data/M72_predictions_for_only_12442/RS_036_MLRC_2022_36.csv saved
4




1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
../data/M72_predictions_for_only_12442/RS_030_MLRC_2022_30.csv
../data/M72_predictions_f

In [11]:
label_file_list_134 = [i for i in glob.glob("../data/M72_predictions_for_only_12442/")]
len(label_file_list_134)

1

In [ ]:
sum = 0
for file_ in label_file_list_134:
    df_ = pd.read_csv(file_)
    # print(len(df_))

    predictions_df = df_.copy()

    # predictions_df = predictions_df[predictions_df['target_predict_label']=='related'][['text','target_predict','target_predict_label']].reset_index(drop=True)
    # predictions_df = predictions_df[['text','target_predict','target_predict_label']]#.reset_index(drop=True)

    sum += len(predictions_df)
sum

12442